In [1]:
#%load_ext autoreload
#%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import tensorflow as tf
import numpy as np
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc
from scipy.fftpack import dct,idct
from keras.optimizers import Adam


sys.path.append('../')
sys.path.append('../models/')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from models.neural_network import NeuralNetwork
from models.cnn import CNN
from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/home/jasjeet/local/miniconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jasjeet/local/miniconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jasjeet/local/miniconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jasjeet/local/miniconda/lib/python3.7/site-packages/ten

In [2]:
#Define params of model
input_shape = (28,28,1)
num_classes = 10

# Load/Create Network and Adversarial data

In [3]:
%%capture one
#Load model from disk
model_name = 'FMNIST-PGD'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = CNN(model_name=model_name, dataset='fashion_mnist',seed=SEED,project=False)
print ('Total Model Params: %d' % model.num_params)
#model.load_model(model_save_path, weights_save_path) 







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
#Shuffle training data
unshuffled_data = model.train_data
unshuffled_labels = model.train_labels
shuffle_idx = np.arange(0,unshuffled_data.shape[0])
np.random.shuffle(shuffle_idx)
shuffled_data = unshuffled_data[shuffle_idx]
shuffled_labels = unshuffled_labels[shuffle_idx]

In [5]:
%%capture three
num_test_samples_per_class = 10
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.val_labels, seed=SEED, num_samples=num_test_samples_per_class)

In [6]:
results = []

In [ ]:
%%capture training
test_dict ={model.input_placeholder: model.val_data[test_indices],model.labels_placeholder: model.val_labels[test_indices]}
global_step = tf.contrib.framework.get_or_create_global_step()
learning_rate = 1e-2
model.train_step = tf.train.AdamOptimizer(learning_rate).minimize(model.training_loss, global_step=global_step)
init = tf.global_variables_initializer()
model.sess.run(init)
batch_size = 128
num_training_steps =10000
for i in range(num_training_steps):
        j = int(i % int(shuffled_data.shape[0]/batch_size))
        x_batch, y_batch = shuffled_data[j*batch_size:j*batch_size + batch_size], shuffled_labels[j*batch_size:j*batch_size + batch_size]

        # Compute Adversarial Perturbations
        x_batch_adv = model.pgd_perturb(x_batch, y_batch)
   
        #create model inputs
        adv_dict={model.input_placeholder: x_batch_adv,model.labels_placeholder: y_batch}
        act_dict={model.input_placeholder: x_batch,model.labels_placeholder: y_batch}

        # Actual training step
        model.sess.run(model.train_step, feed_dict=adv_dict)
    
        if i % 100 == 0:
            adv_accuracy = model.sess.run(model.accuracy, feed_dict=adv_dict)
            act_accuracy = model.sess.run(model.accuracy, feed_dict=act_dict)
            test_acc = model.sess.run(model.accuracy, feed_dict=test_dict)
            results.append((adv_accuracy, act_accuracy, test_acc, i))
            print(adv_accuracy, act_accuracy, test_acc, i)
        
        if i % 1000 == 0 and i > 0:
            model_name = 'FMNIST-PGD-iter-' + str(i)
            model_save_path = '../trained_models/' + model_name + '-model.json'
            weights_save_path = '../trained_models/' + model_name + 'weights'
            model.save_model(model_save_path, weights_save_path)   

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [ ]:
%%capture saving
model_name = 'FMNIST-PGD-iter-' + str(num_training_steps)
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model.save_model(model_save_path, weights_save_path) 

np.save('../data/results-fmnist-pgd', results)

In [ ]:
%%capture loading
model = CNN(model_name=model_name, dataset='fashion_mnist',seed=SEED,project=False)
model.model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
model.load_model(model_save_path, weights_save_path) 
print ('Model Accuracy REG: %.5f' % (model.model.evaluate(model.test_data,model.test_labels)[1]))

In [ ]:
%%capture two
#Get training samples
num_train_samples = 1000
data_indices = model.gen_rand_indices(low=int(model.train_data.shape[0]/2), high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)

In [ ]:
%%capture three
num_test_samples_per_class = 100
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

**GREYBOX ATTACKS**

In [ ]:
%%capture four
import time
#Get Regular, JSMA, DF, and CW test points
reg_data = model.test_data[test_indices]
start_time = time.time()
cwl2_data = model.generate_perturbed_data(model.test_data[test_indices], seed=SEED, perturbation='CW-l2')
print("--- CW-l2: %s seconds ---" % (time.time() - start_time))
start_time = time.time()
df_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='DF', nb_candidate=num_classes)
print("--- DF: %s seconds ---" % (time.time() - start_time))

In [ ]:
%%capture five
np.save('../data/reg_data_fmnist_pgd', reg_data)
np.save('../data/cwl2_data_fmnist_pgd', cwl2_data)
np.save('../data/df_data_fmnist_pgd', df_data)

In [ ]:
%%capture six
jsma_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='JSMA', iterations=100)
print("--- JSMA: %s seconds ---" % (time.time() - start_time))
start_time = time.time()
cwl0_data = model.generate_perturbed_data(model.test_data[test_indices], seed=SEED, perturbation='CW-l0')
print("--- CW: %s seconds ---" % (time.time() - start_time))

In [ ]:
%%capture seven
bim_data = model.generate_perturbed_data(model.test_data[test_indices], seed=SEED, perturbation='BIM')

In [ ]:
%%capture eight
np.save('../data/cwl0_data_fmnist_pgd', cwl0_data)
np.save('../data/bim_data_fmnist_pgd', bim_data)
np.save('../data/jsma_data_fmnist_pgd', jsma_data)

In [17]:
print ('Model Accuracy REG: %.5f' % (model.model.evaluate(model.test_data,model.test_labels)[1]))
print ('Model Accuracy CW-l2: %.5f' % (model.model.evaluate(cwl2_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy DF: %.5f' % (model.model.evaluate(df_data,model.test_labels[test_indices])[1]))

6073/6073 [==============================] - 1s 169us/step
Model Accuracy REG: 0.77820
1000/1000 [==============================] - 0s 49us/step
Model Accuracy CW-l2: 0.75900
1000/1000 [==============================] - 0s 46us/step
Model Accuracy DF: 0.25600


In [18]:
print ('Model Accuracy CW-l0: %.5f' % (model.model.evaluate(cwl0_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy JSMA: %.5f' % (model.model.evaluate(jsma_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM: %.5f' % (model.model.evaluate(bim_data,model.test_labels[test_indices])[1]))

1000/1000 [==============================] - 0s 50us/step
Model Accuracy CW-l0: 0.65100
1000/1000 [==============================] - 0s 49us/step
Model Accuracy JSMA: 0.76600
1000/1000 [==============================] - 0s 48us/step
Model Accuracy BIM: 0.77300
